# Determining Confidence Interval for Average Folk/Country Score with Bootstrapping

First import libraries and connect to database.

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from scipy import stats

In [2]:
# Create a connection to the database
conn = psycopg2.connect("dbname=pitchfork")

In [3]:
cur = conn.cursor()

Query for all folk/country scores.

In [4]:
query = """SELECT score 
FROM reviews 
JOIN genres ON reviews.reviewid = genres.reviewid
WHERE genre = 'folk/country';"""

In [5]:
cur.execute(query)

In [6]:
df_fc_scores = pd.DataFrame(cur.fetchall())
df_fc_scores.columns = [i[0] for i in cur.description]

In [7]:
df_fc_scores.head()

,score
0,6.8
1,7.8
2,7.3
3,8.5
4,7.8


In [8]:
df_fc_scores.shape

(685, 1)

Write function find bootstrapped mean and confidence interval.

In [9]:
def bootstrap_mean_ci(data, n, ci):
    """Take in data, sample size, and confidence interval level and return
    bootstrapped mean of data and confidence interval."""
    sample = np.random.choice(data, size=n)
    bootstrap_samples = []
    bootstrap_sample_means = np.zeros(1000)
    for i in range(1000):
        bootstrap_sample = np.random.choice(sample, size=n)
        bootstrap_samples.append(bootstrap_sample)
        bootstrap_sample_means[i] = bootstrap_sample.mean()
        b_mean = bootstrap_sample_means.mean()
        ordered_bootstrap_sample_means = np.sort(bootstrap_sample_means)
        lower = np.percentile(ordered_bootstrap_sample_means, (1-ci)/2)
        upper = np.percentile(ordered_bootstrap_sample_means, ci+((1-ci)/2))
    return b_mean, lower, upper
    

In [10]:
test = bootstrap_mean_ci(df_fc_scores['score'], 100, 0.95)

In [11]:
print(f'The mean is {test[0]} with a confidence interval of ({test[1]}, {test[2]}).')

The mean is 7.230877 with a confidence interval of (6.8864885000000005, 7.019922).
